In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import numpy as np
import os
import cvlib as cv

In [ ]:
#loading the model
model = load_model("model.h5")

In [ ]:
cam = cv2.VideoCapture(0) #stream video. 0 denotes primary camera. if more cameras are present, 1 or 2 can be given.
classes = ['woman','man']

In [ ]:
while cam.isOpened(): #this method returns true if video capture has been initialised
    ret, frame = cam.read() #read frames. ret stores the bool value, confirming whether the frame is read right.
    
    #cvlib detect_face method detects the face in the image
    face, confidence = cv.detect_face(frame)
    
    #looping through the faces
    for index,f in enumerate(face):
        startx,starty = f[0],f[1]  #startx and starty are x and y coordinates of bottom left vertex of the box.
        endx,endy = f[2],f[3]     ##endx and endy are x and y coordinates of top right vertex of the box.
        
        #drawing a rectangle over the detected face
        cv2.rectangle(frame, (startx,starty), (endx,endy), (0,255,0),2) #2 is the thickness of the rectangle here
        
        #cropping the detected face
        face_crop = np.copy(frame[starty:endy, startx:endx])
        
        
        
        
        #preprocessing the image for the model
        face_crop = cv2.resize(face_crop, (80,80))
        face_crop  = face_crop.astype("float")/255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis =0)
        
        
        verd = model.predict(face_crop)[0] #this gives an array with confidence of each class. confidence for woman at 0th index, and at 1, for man
        idx = np.argmax(verd) #this gives 
        if idx==0:
            label = 'woman'
        else:
            label = 'man'

        label = "{}: {:.2f}%".format(label,verd[idx] * 100)
        
      
        X = int((startx+endx)/2)
        
        cv2.putText(frame,label, (X,endy) , cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0),2)
    
    #displaying image
    cv2.imshow("Detect", frame)
    
    #pressing "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
#release camera and destroy windows
cam.release()
cv2.destroyAllWindows()
    